In [2]:
using DelimitedFiles

In [3]:
using Surrogates

In [4]:
using SurrogatesPolyChaos

In [5]:
using SurrogatesAbstractGPs

In [6]:
using NeuralOperators

In [7]:
using LinearAlgebra

FE(u, v) = norm(u.-v)/norm(v) 

FE (generic function with 1 method)

In [8]:
trainingsets = []
validsets = []
traininglabels = []
validlabels = []

for coarseresolution in 4:5, name in ["fisher", "fourier"]
##loading data
X = readdlm("../data/X_$(name)$(coarseresolution^2)_small.csv", ',')
y = readdlm("../data/y_$(name)$(coarseresolution^2)_small.csv", ',')[:];

Xv = X[:, 1:1024] #valid set
Xtest = X[:, end-1023:end] #test set
Xt = X[:, 1025:end]

yv = y[1:1024] #valid set
ytest = y[end-1023:end] #test set
yt = y[1025:end];

kval = 2^7
N = 256+8*kval

Xt_surrogate = [tuple(Xt[:,i]...) for i=1:size(Xt, 2)][1:N]
yt_surrogate = yt[1:N]
Xv_surrogate = [tuple(Xv[:,i]...) for i=1:size(Xv, 2)]
yv_surrogate = yv
Xtest_surrogate = [tuple(Xtest[:,i]...) for i=1:size(Xtest, 2)]
ytest_surrogate = ytest

push!(trainingsets, Xt_surrogate)
push!(traininglabels, yt_surrogate)

push!(validsets, Xv_surrogate)
push!(validlabels, yv_surrogate);
end

In [18]:

FEs = []
for (Xt_surrogate,
    yt_surrogate,
    Xv_surrogate,
    yv_surrogate) in zip(trainingsets,
    traininglabels,
    validsets,
    validlabels)
    for surrmethod in [RadialBasis, PolynomialChaosSurrogate, AbstractGPSurrogate]
        if surrmethod==AbstractGPSurrogate
            cur_funct = surrmethod(Xt_surrogate, yt_surrogate)
        else
            cur_funct = surrmethod(Xt_surrogate, yt_surrogate,  ones(length(Xt_surrogate[1]))*0., ones(length(Xt_surrogate[1]))*1.)
        end
        preds = cur_funct.(Xv_surrogate);
        FEval = FE(preds, yv_surrogate)
        push!(FEs, FEval)
    end
end

In [19]:
##loading data
X = readdlm("../data/X_maxwell10_small.csv", ',')
y = parse.(Complex{Float64}, readdlm("../data/y_maxwell10_small.csv", ',')[:])

Xv = X[:, 1:1024] #valid set
Xtest = X[:, end-1023:end] #test set
Xt = X[:, 1025:end]

yv = y[1:1024] #valid set
ytest = y[end-1023:end] #test set
yt = y[1025:end];

Xt_surrogate = [tuple(Xt[1:10,i]...,dot(Xt[11:13,i],[0; 0.5; 1])) for i=1:size(Xt, 2)][1:N]
yt_surrogate_r = real.(yt[1:N])
yt_surrogate_i = imag.(yt[1:N])
Xv_surrogate = [tuple(Xv[1:10,i]...,dot(Xv[11:13,i],[0; 0.5; 1])) for i=1:size(Xv, 2)]
yv_surrogate_r = real.(yv)
yv_surrogate_i = imag.(yv);
Xtest_surrogate = [tuple(Xtest[1:10,i]...,dot(Xtest[11:13,i],[0; 0.5; 1])) for i=1:size(Xtest, 2)]
ytest_surrogate_r = real.(ytest)
ytest_surrogate_i = imag.(ytest);

In [22]:
lb = [0.15*ones(10);0.]
ub = [0.85*ones(10);1.]
for surrmethod in [RadialBasis, Kriging, PolynomialChaosSurrogate, AbstractGPSurrogate]
    if surrmethod==AbstractGPSurrogate
        cur_funct_r = surrmethod(Xt_surrogate, yt_surrogate_r)
        cur_funct_i = surrmethod(Xt_surrogate, yt_surrogate_i)
    else
        cur_funct_r = surrmethod(Xt_surrogate, yt_surrogate_r,  ones(length(Xt_surrogate[1]))*0., ones(length(Xt_surrogate[1]))*1.)
        cur_funct_i = surrmethod(Xt_surrogate, yt_surrogate_i,  ones(length(Xt_surrogate[1]))*0., ones(length(Xt_surrogate[1]))*1.)
    end
    preds = cur_funct_r.(Xv_surrogate) .+ 1im .* cur_funct_i.(Xv_surrogate);
    FEval = FE(preds, yv_surrogate_r .+ 1im .* yv_surrogate_i)
    push!(FEs, FEval)
end


In [23]:
vcat([4.5 3.7  5.5 3.8 19]./100,reshape(FEs, (:,5))) # [PEDS, fisher4, fourier4, fisher5, fourier5] x [RadialBasis, Kriging, PolynomialChaosSurrogate, AbstractGPSurrogate]

5×5 Matrix{Any}:
 0.045      0.037      0.055      0.038      0.19
 0.0125316  0.0110112  0.0189032  0.0156031  0.335389
 0.167724   0.114938   0.184185   0.117799   0.55601
 0.0139447  0.0139159  0.0152807  0.0142008  0.721081
 0.0154433  0.0139587  0.0320266  0.0274194  0.365825